In [1]:
#pip install openai==0.28

In [2]:
import openai
import pandas as pd
import json

In [3]:
df = pd.read_excel("C://Users//ashis//OneDrive//Desktop//Bekaert/Cord Construction.xlsx", skiprows=0)
df.rename(columns={'steel cord construction': 'scc'}, inplace=True)
df

,raw data,scc,type,use,lay length,lay direction,target diameter,plus minus,tolerance
0,"2+1x0.30 HT PCR oo/16 -S 0,75 ± 0.06",2+1x0.30,HT,PCR,oo/16,-S,"0,75",±,0.06
1,"2+2x0.25 NT PCR oo/14/14 -SS 0,65 ± 0.05",2+2x0.25,NT,PCR,oo/14/14,-SS,"0,65",±,0.05
2,"2+2x0.25 HT PCR oo/14/14 -SS 0,65 ± 0.05",2+2x0.25,HT,PCR,oo/14/14,-SS,"0,65",±,0.05
3,"2+2x0.30 HT PCR oo/16/16 -SS 0,78 ± 0.06",2+2x0.30,HT,PCR,oo/16/16,-SS,"0,78",±,0.06
4,"2+4x0.17 UT PCR oo/10/10 -SS 0,48 ± 0.04",2+4x0.17,UT,PCR,oo/10/10,-SS,"0,48",±,0.04
...,...,...,...,...,...,...,...,...,...
71,7x(3+9x0.30)+0.20 HT OTR 12.5/18/32/5 ZZSSZS 4...,7x(3+9x0.30)+0.20,HT,OTR,12.5/18/32/5,ZZSSZS,"4,11",±,0.24
72,"7x7x0.22+0.15 NT OTR 12.5/20/5 SZS 2,24 ± 0.11",7x7x0.22+0.15,NT,OTR,12.5/20/5,SZS,"2,24",±,0.11
73,"7x7x0.22+0.15 HT OTR 12.5/20/5 SZS 2,24 ± 0.11",7x7x0.22+0.15,HT,OTR,12.5/20/5,SZS,"2,24",±,0.11
74,"7x7x0.25+0.15 HT OTR 12.5/20/5 SZS 2,52 ± 0.13",7x7x0.25+0.15,HT,OTR,12.5/20/5,SZS,"2,52",±,0.13


In [4]:
df.drop(columns=['raw data','plus minus','lay length', 'lay direction'], inplace=True)
df

,scc,type,use,target diameter,tolerance
0,2+1x0.30,HT,PCR,"0,75",0.06
1,2+2x0.25,NT,PCR,"0,65",0.05
2,2+2x0.25,HT,PCR,"0,65",0.05
3,2+2x0.30,HT,PCR,"0,78",0.06
4,2+4x0.17,UT,PCR,"0,48",0.04
...,...,...,...,...,...
71,7x(3+9x0.30)+0.20,HT,OTR,"4,11",0.24
72,7x7x0.22+0.15,NT,OTR,"2,24",0.11
73,7x7x0.22+0.15,HT,OTR,"2,24",0.11
74,7x7x0.25+0.15,HT,OTR,"2,52",0.13


In [5]:
shuffled_df = df.sample(frac=1).reset_index(drop=True)
shuffled_df

,scc,type,use,target diameter,tolerance
0,4x0.35Betru,ST,LTR/SUV,"0,85",0.04
1,0.22+18x0.20CC,UT,TBR,"1,02",0.05
2,0.22+6+12x0.20,HT,TBR,"1,02",0.05
3,4+3x0.35Betru,ST,TBR,"1,17",0.07
4,4+6x0.35Betru,HT,TBR,"1,41",0.07
...,...,...,...,...,...
71,5x0.35HighImpact,HI,TBR,"1,19",0.06
72,0.22+18x0.20CC,NT,TBR,"1,02",0.05
73,2x0.30,HT,PCR,"0,60",0.03
74,3x0.20+6x0.35,NT,TBR,"1,13",0.06


In [6]:
test_df = shuffled_df.tail(22)
test_df.head()

,scc,type,use,target diameter,tolerance
54,3+9+9x3x0.25+0.15,HT,OTR,"2,34",0.12
55,3x0.20+9x0.175CC,NT,LTR/SUV,"0,75",0.04
56,0.365+6x0.35Betru,HT,TBR,"1,08",0.05
57,0.20+18x0.175CC,HT,TBR,"0,90",0.05
58,3x4x0.22,HE,TBR,"1,14",0.07


In [7]:
train_df = shuffled_df.head(54)
train_df.tail()

,scc,type,use,target diameter,tolerance
49,7x(3+9+15x0.22)+0.20,HT,OTR,"4,29",0.21
50,2+4x0.17,UT,PCR,"0,48",0.04
51,2+2x0.32,HT,LTR/SUV,"0,83",0.07
52,0.20+18x0.175CC,NT,TBR,"0,90",0.05
53,4x4x0.22,HE,TBR,"1,32",0.08


In [8]:
print(train_df.columns)

Index(['scc', 'type', 'use', 'target diameter', 'tolerance'], dtype='object')


In [9]:
scc_list = []
for i in train_df['scc']:
    scc_list.append(i)
scc_str = ', '.join(scc_list)

In [18]:
prompt = f"""I require the generation of precisely 10 additional cord constructions based solely on the provided steel cord input. The input steel cord constructions are as follows: {scc_str}. It is imperative to strictly adhere to the production of exactly 10 outputs, without any repetitions of the input cord constructions. The output should be exclusively in JSON format. Please ensure that the output comprises solely of JSON data as a fact, with no additional text."""

In [11]:
api_key = api_key # Paste API Key
api_base = api_base # Paste API Base
api_type = 'azure'
api_version = '2023-05-15'

In [19]:
openai.api_key = api_key
openai.api_base = api_base
openai.api_type = api_type
openai.api_version = api_version

request = openai.ChatCompletion.create(
            engine="Development",
            model="gpt-3.5-turbo",
            messages=[
                {"role": "user", "content": prompt},
            ])
result = request['choices'][0]['message']['content']
print(result)
try:
    new_construction = json.loads(result)

    new_cons_df = pd.DataFrame(new_construction.items(), columns=['i', 'scc'])
    new_cons_df.drop(columns=['i'], inplace=True)
except:
    print("Re-run")
new_cons_df

{"cord1": "2+3x0.25Betru",
 "cord2": "0.25+12x0.22CC",
 "cord3": "5+3x0.35Betru",
 "cord4": "4x0.40Betru",
 "cord5": "0.30+9x0.25CC",
 "cord6": "4x0.30Betru+0.25",
 "cord7": "0.22+12x0.20CC+0.15",
 "cord8": "3+9x0.30Betru+0.25",
 "cord9": "3x0.27+9x0.225CC",
 "cord10": "3x0.24+12x0.20"}


,scc
0,2+3x0.25Betru
1,0.25+12x0.22CC
2,5+3x0.35Betru
3,4x0.40Betru
4,0.30+9x0.25CC
5,4x0.30Betru+0.25
6,0.22+12x0.20CC+0.15
7,3+9x0.30Betru+0.25
8,3x0.27+9x0.225CC
9,3x0.24+12x0.20
